In [ ]:
import pandas as pd
import os
import datetime
from time import sleep

from scrapy.selector import Selector
from scrapy.http import HtmlResponse

from selenium import webdriver

#CHANGE THIS WITH THE RIGHT PATH!
chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe"
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
from selenium.webdriver.common.keys import Keys
import re


def get_url(job_portal,keywords):
    """
    this function requires 
    1)the start page of the job portal
    2)keywords
    
    it will obtain all the major/total links for the keywords
    take note for mycareersfuture.sg,1 page has 20 listings and
    the URL is a running runner hence we can truncate the last number and add our own counter
    to save our pages
    i.e https://www.mycareersfuture.sg/search?search=analytics&sortBy=new_posting_date&page=0
    
    """
    
    driver = webdriver.Chrome(executable_path=chromedriver)
    driver.get(job_portal) #point chrome to base URL
    sleep(3)
    inputElement = driver.find_element_by_xpath("//input[@type='text']") #get the input box
    inputElement.send_keys(keywords) #type 'job title'
    inputElement.send_keys(Keys.ENTER) #press enter
    sleep(3)
    
    url=driver.current_url #save the URL to truncate later
    sleep(3)
    HTML=driver.page_source #save the page
    driver.close()
    #extra total number of pages for the keyword listing
    #do some data cleaning
    num_jobs_dirty = Selector(text=HTML).xpath('//span[contains(@id,"search-results")]/div/text()').extract()
    
    num_jobs_dirty=num_jobs_dirty[0].replace(',','').split(' ')
    num_jobs=int(num_jobs_dirty[0])
    
    import math
    num_jobs = math.ceil(num_jobs/20) #20 listings per page and round up
    url_loop = re.sub('\d{1,2}$','',url) #truncate the last digit of the URL
    
    scrape_url=[] # loop to get all the running URLs to scrape
    for num_job in range(num_jobs):
        scrape_url.append(url_loop+str(num_job))

    return scrape_url

### I have re-run the scraping using only 2 keywords as running the full keyword is too long .

### BUT will be using the full keywords for submission. This is to ensure that the notebook is good order. 

### Thank you for your understanding

In [3]:
job_portal='https://www.mycareersfuture.sg' #where do you want to scrape today?

#keywords to input to the search bar
keywords = ['data scientist','data consultant']

#keywords = ['data analyst','business analyst','data scientist','machine learning','analytics','visualisation',
#           'marketing analytics','data strategist','data consultant','business intelligence']


total_url=[]  #initialisation
for keyword in keywords: #this for loop will loop through all the keywords for the get_url function
    loop_url=get_url(job_portal,keyword) #input 2 arguments
    total_url+=loop_url #save all the URL by appending to the list

In [4]:
total_url

['https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=0',
 'https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=1',
 'https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=2',
 'https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=3',
 'https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=4',
 'https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=5',
 'https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=6',
 'https://www.mycareersfuture.sg/search?search=data%20consultant&sortBy=new_posting_date&page=0']

In [5]:
#write the major url to csv
major_url=pd.DataFrame(total_url, columns=["major_url"])
major_url.to_csv('.\datasets\major_url.csv', index=False)

In [6]:
total_url=pd.read_csv('.\datasets\major_url.csv')

In [7]:
def get_individual_url(list_of_all_url):
#this function will accept a list and will webscrape a list of all individual URLs of each job
    url_df = pd.DataFrame()
    driver = webdriver.Chrome(executable_path=chromedriver)
    for element in list_of_all_url:
        driver.get(element)
        print(element)
        sleep(4)
        HTML=driver.page_source
        list_of_url = Selector(text=HTML).xpath('//a[contains(@class,"JobCard")]/@href').extract()
        
        ind_url=[]
        base_url='https://www.mycareersfuture.sg'
        
        
        for url in list_of_url:
                url = [ base_url+url ] 
                temp = pd.DataFrame(url,columns=["ind_url"])
                url_df = url_df.append(temp)
        url_df.reset_index(drop=True,inplace=True)   
        url_df.to_csv('.\datasets\ind_link.csv',index=False)
        
    return url_df

In [8]:
major_url

,major_url
0,https://www.mycareersfuture.sg/search?search=d...
1,https://www.mycareersfuture.sg/search?search=d...
2,https://www.mycareersfuture.sg/search?search=d...
3,https://www.mycareersfuture.sg/search?search=d...
4,https://www.mycareersfuture.sg/search?search=d...
5,https://www.mycareersfuture.sg/search?search=d...
6,https://www.mycareersfuture.sg/search?search=d...
7,https://www.mycareersfuture.sg/search?search=d...


In [10]:
temp=get_individual_url(major_url['major_url'])
temp

https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=0
https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=1
https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=2
https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=3
https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=4
https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=5
https://www.mycareersfuture.sg/search?search=data%20scientist&sortBy=new_posting_date&page=6
https://www.mycareersfuture.sg/search?search=data%20consultant&sortBy=new_posting_date&page=0


,ind_url
0,https://www.mycareersfuture.sg/job/data-scient...
1,https://www.mycareersfuture.sg/job/director-da...
2,https://www.mycareersfuture.sg/job/senior-anal...
3,https://www.mycareersfuture.sg/job/data-scient...
4,https://www.mycareersfuture.sg/job/2019-market...
5,https://www.mycareersfuture.sg/job/data-scient...
6,https://www.mycareersfuture.sg/job/data-scient...
7,https://www.mycareersfuture.sg/job/data-scient...
8,https://www.mycareersfuture.sg/job/data-scient...
9,https://www.mycareersfuture.sg/job/rd-project-...


In [11]:
#write the major url to csv
ind_url_df=pd.DataFrame(temp, columns=["ind_url"])
ind_url_df.to_csv('.\datasets\ind_url.csv', index=False)

In [12]:
ind_url_df=pd.read_csv('.\datasets\ind_url.csv')

In [13]:
ind_url_df.head()

,ind_url
0,https://www.mycareersfuture.sg/job/data-scient...
1,https://www.mycareersfuture.sg/job/director-da...
2,https://www.mycareersfuture.sg/job/senior-anal...
3,https://www.mycareersfuture.sg/job/data-scient...
4,https://www.mycareersfuture.sg/job/2019-market...


In [14]:
ind_links =pd.read_csv('.\datasets\ind_link.csv')
for link in ind_links.ind_url.values:
    print(link)    

https://www.mycareersfuture.sg/job/data-scientist-perm-investment-banking-david-goliath-b50d317b1910e630bada58d6b25e03d0
https://www.mycareersfuture.sg/job/director-data-engineering-analytics-product-development-data-services-mastercard-asiapacific-412c2c9c48e86b8bea097b7234f810f6
https://www.mycareersfuture.sg/job/senior-analyst-new-product-solution-development-data-science-engineering-data-services-mastercard-asiapacific-b1c0120d5ed312778bf98d0340e41d87
https://www.mycareersfuture.sg/job/data-scientist-moloco-singapore-b805aedf0b578093616e96d94961ca4b
https://www.mycareersfuture.sg/job/2019-market-data-analyst-bloomberg-singapore-048c42b9d5bd3e02fd60f6a69576384a
https://www.mycareersfuture.sg/job/data-scientist-eyeota-fc5e0701c730a2c2c7d5adb730b8683d
https://www.mycareersfuture.sg/job/data-scientist-experian-singapore-549714719c70d96e38c7ca55eae9b4dc
https://www.mycareersfuture.sg/job/data-scientist-observational-pragmatic-research-institute-ffbd8700991fc53857c6f73cc5f8987a
https://w

In [16]:
driver = webdriver.Chrome(executable_path=chromedriver)

for number,link in enumerate(ind_links.ind_url.values):
    print(number)
    driver.get(link)
    sleep(3)
    HTML=driver.page_source
    
    try:
        company = Selector(text=HTML).xpath('//p[@name="company"]/text()').extract()[0]
        company.replace('\'','')
    except:
        pass
    
    try:
        job_id = Selector(text=HTML).xpath('//div[contains(@class,"jobInfo")]/span/text()').extract()[0]
        job_id.replace('\'','')
    except:
        pass
    
    try:
        title = Selector(text=HTML).xpath('//h1[@id="job_title"]/text()').extract()[0]
        title.replace('\'','')
    except:
        pass
    
    try:
        location = Selector(text=HTML).xpath('//a[@href="#location_map"]/text()').extract()[0]
        location.replace('\'','')
    except:
        pass

    try:
        employment_type = Selector(text=HTML).xpath('//p[@id="employment_type"]/text()').extract()[0]
        employment_type.replace('\'','')
    except:
        pass
    
    try:
        level = Selector(text=HTML).xpath('//p[@id="seniority"]/text()').extract()[0]
        level.replace('\'','')
    except:
        pass
    
    try:
        exp = Selector(text=HTML).xpath('//p[@id="min_experience"]/text()').extract()[0]
        exp = re.sub('\d+','', exp)
    except:
        pass
    
    try:
        category = Selector(text=HTML).xpath('//p[@id="job-categories"]/text()').extract()[0]
        category.replace('/','')
    except:
        pass
    
    try:
        salary_min = Selector(text=HTML).xpath('//section[contains(@class,"salary")]//span/text()').extract()[0]
        salary_min = re.sub('\D+','', salary_min)
    except:
        pass
    
    try:
        salary_max = Selector(text=HTML).xpath('//section[contains(@class,"salary")]//span/text()').extract()[2]
        salary_max = re.sub('\D+','', salary_max)
    except:
        pass
    
    try:
        salary_time = Selector(text=HTML).xpath('//span[contains(@class,"salary_type")]//text()').extract()[0]
    except:
        pass
    
    try:
        date_close = Selector(text=HTML).xpath('//span[contains(@id,"expiry_date")]/text()').extract()[0]
        date_close.replace('Closing on ','')
    except:
        pass
    
    try:
        roles = Selector(text=HTML).xpath('//div[contains(@id,"description-content")]//text()').extract()
        roles=[ role.strip() for role in roles if role.strip()!='' ]
        clean_role = ' '.join(roles)
    except:
        pass
    
    try:
        reqs = Selector(text=HTML).xpath('//div[contains(@id,"requirements")]//text()').extract()
        reqs=[req.strip() for req in reqs if req.strip()!='' ]
        clean_req = ' '.join(reqs)
    except:
        pass
    
    try:
        skills = Selector(text=HTML).xpath('//div[contains(@class,"multi-pill-button")]//label/text()').extract()
        clean_skill = ','.join(skills)
    except:
        pass
    
    url=link
    date_scrape = datetime.datetime.today().strftime("%d-%m-%Y")    
        
    #write to df
    webscrape = [job_id,company,title,location,employment_type,level,category,salary_min,category,salary_max,
                 salary_time,exp,date_close,clean_role,clean_req,clean_skill,url,date_scrape]
    
    job_columns = ['job_id','company','title','location','employment_type','level','category','salary_min','category',
                   'salary_max','salary_time','exp','date_close','clean_role','clean_req','skills','url','date_scrape']

    temp = pd.DataFrame.from_records(data=[webscrape],columns=[job_columns])
    df = df.append(temp,ignore_index=True)
    df.reset_index(drop=True,inplace=True)   
    df.to_csv('.\datasets\df.csv',index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
